In [4]:
import pandas as pd
import numpy as np
import xarray as xr
import cdsapi 

# viz
import matplotlib.pyplot as plt
import plotly.express as px
import cartopy
import cartopy.crs as ccrs
from cartopy.io import shapereader
import plotly.graph_objects as go

import glob
import sys
import os

import geopandas as gpd
import rioxarray
from shapely.geometry import mapping

In [122]:
# Get a list of all CSV files in the directory that start with "ERA5_mean"
csv_files = glob.glob('../../data/UNSEEN/wildfire/cems/preprocessed/ERA5_mean*.csv')

# Initialize an empty DataFrame
df = pd.DataFrame()

# Loop through the list of CSV files
for file in csv_files:
    # Extract the state abbreviation from the filename
    state = os.path.basename(file).split('_')[-1].split('.')[0]
    # Read the CSV file into a DataFrame
    data = pd.read_csv(file)
    
    # Add a new column to the DataFrame for the state abbreviation
    data['state'] = state
    
    data = data[data['Year'] != 2024]

    #data = data[data['Year'] > 1983]

    
    # Append the DataFrame to the main DataFrame
    df = pd.concat([df, data], ignore_index=True)

# Create a line plot with 'year' on the x-axis and 'mean_fire_weather_index' on the y-axis
fig = px.line(df, x='Year', y='mean_fire_weather_index', color='state', title='Mean Fire Weather Index Over Time by State')
fig.show()

In [52]:
# Read the CSV file into a DataFrame
extremes = pd.read_csv('../../data/UNSEEN/wildfire/cems/preprocessed/ERA5_extreme_fwi_days_by_month_TX.csv')

# Filter out data from years before 1984 and from the year 2024
extremes = extremes[(extremes['Year'] >= 1983) & (extremes['Year'] != 2024)]

# Reshape the DataFrame from wide format to long format
extremes_long = extremes.melt(id_vars='Year', var_name='Month', value_name='extreme_days')

# Convert the 'Month' column to a categorical type with the correct order
extremes_long['Month'] = pd.Categorical(extremes_long['Month'], categories=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], ordered=True)

# Sort the DataFrame by 'Year' and 'Month'
extremes_long = extremes_long.sort_values(['Year', 'Month'])

# Create a line plot with 'Year' on the x-axis and 'extreme_days' on the y-axis, and one line per 'Month'
fig = px.line(extremes_long, x='Year', y='extreme_days', color='Month', title='Fire Risk is Going up in the Spring in NM')
fig.update_layout(autosize=False, width=800)

fig.show()

In [53]:
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression

# Read the CSV file into a DataFrame
extremes = pd.read_csv('../../data/UNSEEN/wildfire/cems/preprocessed/ERA5_extreme_fwi_days_by_month_NM.csv')

# Filter out data from years before 1984 and from the year 2024
extremes = extremes[(extremes['Year'] >= 1984) & (extremes['Year'] != 2024)]

# Reshape the DataFrame from wide format to long format
extremes_long = extremes.melt(id_vars='Year', var_name='Month', value_name='extreme_days')

# Convert the 'Month' column to a categorical type with the correct order
extremes_long['Month'] = pd.Categorical(extremes_long['Month'], categories=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], ordered=True)

# Filter the data to only include the months of May, June, and July
extremes_long = extremes_long[extremes_long['Month'].isin(['Apr', 'May', 'Jun'])]

# Group the data by year and sum the 'extreme_days' for each year
extremes_sum = extremes_long.groupby('Year')['extreme_days'].sum().reset_index()

# Create a line plot with 'Year' on the x-axis and 'extreme_days' on the y-axis
fig = px.line(extremes_sum, x='Year', y='extreme_days', title='Spring Fire Risk is Growing in New Mexico<br><sub>Instances of Extreme Fire Weather in April, May, June</sub>')

# Fit a linear regression model to the summed data
model = LinearRegression()
model.fit(extremes_sum[['Year']], extremes_sum['extreme_days'])

# Predict the y values
y_pred = model.predict(extremes_sum[['Year']])

# Add a scatter plot with a trend line for the summed data
fig.add_trace(go.Scatter(x=extremes_sum['Year'], y=y_pred, mode='lines', showlegend=False))

# Remove the y-axis title
fig.update_yaxes(title_text='')

fig.update_layout(autosize=False, width=600)

fig.show()

In [46]:
fig.write_html('../../figures/UNSEEN_wildfire/nm_spring_extreme_fwi.html')
fig.write_image('../../figures/UNSEEN_wildfire/nm_spring_extreme_fwi.png')

In [54]:
import glob
import os
import pandas as pd
import plotly.express as px

# Get a list of all CSV files in the directory that end with "_ERA5_extreme_fwi_days_by_month.csv"
csv_files = glob.glob('../../data/UNSEEN/wildfire/cems/preprocessed/ERA5_extreme_fwi_days_by_month_??.csv')

# Initialize an empty DataFrame
df = pd.DataFrame()

# Loop through the list of CSV files
for file in csv_files:
    # Extract the state abbreviation from the filename
    state = os.path.basename(file).split('_')[-1].split('.')[0]
    
    # Read the CSV file into a DataFrame
    data = pd.read_csv(file)
    
    # Add a new column to the DataFrame for the state abbreviation
    data['state'] = state
    
    # Select only the 'Year', 'Oct', and 'state' columns, and rename 'Oct' to 'extreme_fire_weather'
    data = data[['Year', 'Apr', 'state']].rename(columns={'Apr': 'extreme_fire_weather'})

    data = data[(data['Year'] >= 1984) & (data['Year'] != 2024)]

    
    # Append the DataFrame to the main DataFrame
    df = pd.concat([df, data], ignore_index=True)

# Create a line plot with 'Year' on the x-axis and 'extreme_fire_weather' on the y-axis
fig = px.line(df, x='Year', y='extreme_fire_weather', color='state', title='Number of Instances of Extreme Fire Weather in April Over Time by State')
fig.show()

In [ ]:
fig.write_html('../../figures/UNSEEN/wildfire/extreme_fireweather_april.html')
fig.write_image('../../figures/UNSEEN/wildfire/extreme_fireweather_april.png')

In [66]:
# Get a list of all CSV files in the directory that match the name format
csv_files = glob.glob('../../data/UNSEEN/wildfire/cems/preprocessed/ERA5_extreme_fwi_days_??.csv')

# Initialize an empty DataFrame
df = pd.DataFrame()

# Loop through the list of CSV files
for file in csv_files:
    # Extract the state abbreviation from the filename
    state = os.path.basename(file).split('_')[-1].split('.')[0]
    
    # Read the CSV file into a DataFrame
    data = pd.read_csv(file)
    
    # Add a new column to the DataFrame for the state abbreviation
    data['state'] = state

    data = data[(data['year'] >= 1984) & (data['year'] != 2024)]

    
    # Select only the 'Year', 'p995_count', and 'state' columns
    data = data[['year', 'p995_count', 'state']]
    
    # Append the DataFrame to the main DataFrame
    df = pd.concat([df, data], ignore_index=True)

# Create a line plot with 'Year' on the x-axis and 'p995_count' on the y-axis
fig = px.line(df, x='year', y='p995_count', color='state', title='Number of Instances of Extreme Fire Weather Over Time by State')

#fig.update_layout(autosize=False, width=800)

fig.show()

In [75]:
import statsmodels.api as sm

data = pd.read_csv('../../data/UNSEEN/wildfire/cems/preprocessed/ERA5_extreme_fwi_CO.csv')

# Create a line plot with 'year' on the x-axis and 'p99_count' on the y-axis
fig = px.line(data, x='year', y='p99_count', title='Extreme Fire Risk on the Rise in Colorado<br><sub>Instances of Extreme Fire Weather (>99 percentile)</sub>')

# Fit an ordinary least squares regression model to the data
X = sm.add_constant(data['year'])  # Add a constant term to the predictor
model = sm.OLS(data['p99_count'], X)
results = model.fit()

# Get the fitted values from the model
data['trend'] = results.fittedvalues

# Add a trend line to the plot
fig.add_trace(
    go.Scatter(
        x=data['year'],
        y=data['trend'],
        mode='lines',
        marker=dict(color="red"),
        name='Trend',
        showlegend=False
    )
)

fig.update_layout(autosize=False, width=600)
fig.update_yaxes(title_text='')
fig.update_xaxes(title_text='')


fig.show()

In [77]:
fig.write_html('../../figures/UNSEEN_wildfire/co_extreme_fireweather.html')
fig.write_image('../../figures/UNSEEN_wildfire/co_extreme_fireweather.png')

In [93]:
# canada

color_dict = {'Manitoba': '#636EFA', 'British Columbia': '#EF553B', 'Saskatchewan': '#00CC96', 'Alberta': '#AB63FA', 'Ontario': '#FFA15A', 'Quebec': '#19D3F3'}


In [9]:
# Canada

# Get a list of all CSV files in the directory that start with "ERA5_mean"
csv_files = glob.glob('../../data/UNSEEN/wildfire/cems/preprocessed/canada/ERA5_mean*.csv')

# Initialize an empty DataFrame
df = pd.DataFrame()

# Loop through the list of CSV files
for file in csv_files:
    province = os.path.basename(file).split('_')[-1].split('.')[0]
    
    data = pd.read_csv(file)
    
    data['province'] = province
    
    df = pd.concat([df, data], ignore_index=True)

# Calculate a 3-year rolling mean of 'mean_fire_weather_index'
df['rolling_mean'] = df.groupby('province')['mean_fire_weather_index'].rolling(3).mean().reset_index(0, drop=True)

# Filter the DataFrame
df = df[(df['Year'] >= 2003) & (df['Year'] != 2024)]

# Create a line plot with 'Year' on the x-axis and 'rolling_mean' on the y-axis
fig = px.line(df, x='Year', y='rolling_mean', color='province', 
    title='Fire Risk in Saskatchewan and Alberta Leading the Pack<br><sub>Rolling mean Fire Weather Index</sub>',
    color_discrete_map=color_dict)

# Update the x-axis labels
fig.update_xaxes(
    tickvals=list(range(2023, df['Year'].min() - 1, -5)),
    ticktext=[str(year) for year in range(2023, df['Year'].min() - 1, -5)]
)

fig.update_yaxes(title_text='')
fig.update_xaxes(title_text='')
fig.update_layout(autosize=False, width=800)
fig.update_layout(showlegend=False)

# Get the 2023 values for each province
values_2023 = df[df['Year'] == 2023].sort_values('rolling_mean', ascending=False)

# Add an annotation for each province next to its 2023 value
for i in range(len(values_2023)):
    fig.add_annotation(
        x=1,  # Adjust this value to move the annotations to the right
        y=values_2023['rolling_mean'].iloc[i],
        text=values_2023['province'].iloc[i],
        showarrow=False,
        font=dict(
            size=10,
            color="black"
        ),
        xref="paper",  # Use the relative coordinates of the plot area
        yref="y",  # Use the data coordinates for the y-axis
        xanchor="left",  # Align the text to the left
        yanchor="middle"  # Align the text to the middle vertically
    )

fig.show()

In [163]:
fig.write_html('../../figures/UNSEEN_wildfire/ca_mean_fwi_10_year.html')
fig.write_image('../../figures/UNSEEN_wildfire/ca_mean_fwi_10_year.png')

In [133]:
# Calculate the average 'mean_fire_weather_index' for the period '1982-01-01' to '2022-12-31' for each province
average_1982_2022 = df[(df['Year'] >= 1982) & (df['Year'] <= 2022)].groupby('province')['mean_fire_weather_index'].mean()

# Get the 'mean_fire_weather_index' for 2023 for each province
mean_2023 = df[df['Year'] == 2023].set_index('province')['mean_fire_weather_index']

# Calculate the absolute difference
absolute_difference = (mean_2023 - average_1982_2022).abs()

# Calculate the percentage difference
percentage_difference = (mean_2023 - average_1982_2022).abs() / average_1982_2022 * 100

# Print the absolute difference and the percentage difference
print('Absolute difference:\n', absolute_difference)
print('\nPercentage difference:\n', percentage_difference)

Absolute difference:
 province
Alberta             1.979894
British Columbia    1.043648
Manitoba            0.544118
Ontario             0.837024
Quebec              0.672586
Saskatchewan        2.058897
Name: mean_fire_weather_index, dtype: float64

Percentage difference:
 province
Alberta             35.665122
British Columbia    43.375668
Manitoba            19.396902
Ontario             33.959007
Quebec              49.441741
Saskatchewan        38.332001
Name: mean_fire_weather_index, dtype: float64


In [115]:
fig.write_html('../../figures/UNSEEN_wildfire/ca_mean_fwi.html')
fig.write_image('../../figures/UNSEEN_wildfire/ca_mean_fwi.png')

In [168]:
# Get a list of all CSV files in the directory that match the name format
csv_files = glob.glob('../../data/UNSEEN/wildfire/cems/preprocessed/canada/ERA5_extreme_fwi_days_*.csv')

# Initialize an empty DataFrame
df = pd.DataFrame()

# Loop through the list of CSV files
for file in csv_files:
    # Extract the state abbreviation from the filename
    province = os.path.basename(file).split('_')[-1].split('.')[0]
    
    # Read the CSV file into a DataFrame
    data = pd.read_csv(file)
    
    # Add a new column to the DataFrame for the state abbreviation
    data['province'] = province
    
    # Select only the 'Year', 'p995_count', and 'state' columns
    data = data[['Year', 'p90_count', 'province']]
    
    # Append the DataFrame to the main DataFrame
    df = pd.concat([df, data], ignore_index=True)

df['rolling_mean'] = df.groupby('province')['p90_count'].rolling(3).mean().reset_index(0, drop=True)
df = df[(df['Year'] >= 2003) & (df['Year'] != 2024)]

# Create a line plot with 'Year' on the x-axis and 'p995_count' on the y-axis

fig = px.line(df, x='Year', y='rolling_mean', color='province', 
    title='Extreme Fire Risk<br><sub>Occurences of FWI above 90th percentile</sub>',
    color_discrete_map=color_dict)

fig.update_yaxes(title_text='')
fig.update_xaxes(title_text='')
fig.update_layout(autosize=False, width=800)
fig.update_layout(showlegend=False)

# Get the 2023 values for each province
values_2023 = df[df['Year'] == 2023].sort_values('rolling_mean', ascending=False)

# Add an annotation for each province next to its 2023 value
for i in range(len(values_2023)):
    fig.add_annotation(
        x=1,  # Adjust this value to move the annotations to the right
        y=values_2023['rolling_mean'].iloc[i],
        text=values_2023['province'].iloc[i],
        showarrow=False,
        font=dict(
            size=10,
            color="black"
        ),
        xref="paper",  # Use the relative coordinates of the plot area
        yref="y",  # Use the data coordinates for the y-axis
        xanchor="left",  # Align the text to the left
        yanchor="middle"  # Align the text to the middle vertically
    )

# Update the x-axis labels
fig.update_xaxes(
    tickvals=list(range(2023, df['Year'].min() - 1, -5)),
    ticktext=[str(year) for year in range(2023, df['Year'].min() - 1, -5)]
)

fig.show()

In [169]:
fig.write_html('../../figures/UNSEEN_wildfire/ca_extreme_fwi.html')
fig.write_image('../../figures/UNSEEN_wildfire/ca_extreme_fwi.png')

In [95]:
# Get a list of all CSV files in the directory that match the name format
csv_files = glob.glob('../../data/UNSEEN/wildfire/cems/preprocessed/canada/ERA5_extreme_fwi_days_*.csv')

# Initialize an empty DataFrame
df = pd.DataFrame()

# Loop through the list of CSV files
for file in csv_files:
    # Extract the state abbreviation from the filename
    province = os.path.basename(file).split('_')[-1].split('.')[0]
    
    # Read the CSV file into a DataFrame
    data = pd.read_csv(file)
    
    # Add a new column to the DataFrame for the state abbreviation
    data['province'] = province
    
    # Select only the 'Year', 'p995_count', and 'state' columns
    data = data[['Year', 'p90_count', 'province']]
    
    # Append the DataFrame to the main DataFrame
    df = pd.concat([df, data], ignore_index=True)

#df['rolling_mean'] = df.groupby('province')['p90_count'].rolling(3).mean().reset_index(0, drop=True)
df = df[(df['Year'] != 2024)]

# Create a line plot with 'Year' on the x-axis and 'p995_count' on the y-axis

fig = px.line(df, x='Year', y='p90_count', color='province', 
    title='Extreme Fire Risk<br><sub>Occurences of FWI above 90th percentile</sub>',
    color_discrete_map=color_dict)

fig.update_yaxes(title_text='')
fig.update_xaxes(title_text='')
fig.update_layout(autosize=False, width=800)
#fig.update_layout(showlegend=False)

# Get the 2023 values for each province
#values_2023 = df[df['Year'] == 2023].sort_values('rolling_mean', ascending=False)

# # Add an annotation for each province next to its 2023 value
# for i in range(len(values_2023)):
#     fig.add_annotation(
#         x=1,  # Adjust this value to move the annotations to the right
#         y=values_2023['rolling_mean'].iloc[i],
#         text=values_2023['province'].iloc[i],
#         showarrow=False,
#         font=dict(
#             size=10,
#             color="black"
#         ),
#         xref="paper",  # Use the relative coordinates of the plot area
#         yref="y",  # Use the data coordinates for the y-axis
#         xanchor="left",  # Align the text to the left
#         yanchor="middle"  # Align the text to the middle vertically
#     )

# Update the x-axis labels
fig.update_xaxes(
    tickvals=list(range(2023, df['Year'].min() - 1, -5)),
    ticktext=[str(year) for year in range(2023, df['Year'].min() - 1, -5)]
)

fig.show()

In [118]:
# Get a list of all CSV files in the directory that end with "_ERA5_extreme_fwi_days_by_month.csv"
csv_files = glob.glob('../../data/UNSEEN/wildfire/cems/preprocessed/canada/ERA5_extreme95_fwi_days_by_month_*.csv')

# Initialize an empty DataFrame
df = pd.DataFrame()

# Loop through the list of CSV files
for file in csv_files:
    # Extract the state abbreviation from the filename
    province = os.path.basename(file).split('_')[-1].split('.')[0]
    
    # Read the CSV file into a DataFrame
    data = pd.read_csv(file)
    
    # Add a new column to the DataFrame for the state abbreviation
    data['province'] = province
    
    # Select only the 'Year', 'Oct', and 'state' columns, and rename 'Oct' to 'extreme_fire_weather'
    data = data[['Year', 'May', 'province']].rename(columns={'May': 'extreme_fire_weather'})

    data = data[(data['Year'] != 2024)]
#(data['Year'] >= 1984) & 
    
    # Append the DataFrame to the main DataFrame
    df = pd.concat([df, data], ignore_index=True)

# Create a line plot with 'Year' on the x-axis and 'extreme_fire_weather' on the y-axis
fig = px.line(df, x='Year', y='extreme_fire_weather', color='province', title='Number of Instances of Extreme Fire Weather in May Over Time by Province')
fig.show()

In [1]:
from tools import get_shape_ca, mask_xr_dataset
import geopandas as gpd

from shapely.geometry import mapping

ca_shapefile = gpd.read_file('../../data/shapefiles/canada/lpr_000b16a_e.shp')

shape = get_shape_ca(ca_shapefile, "British Columbia")
province_data_era5 = mask_xr_dataset(ERA5,shape)

NameError: name 'ERA5' is not defined

In [36]:
ERA5 = xr.open_mfdataset('../../data/UNSEEN/wildfire/cems/ERA5/ERA5_????.nc',combine='by_coords')
ERA5

<xarray.Dataset> Size: 1GB
Dimensions:     (time: 18687, latitude: 85, longitude: 173)
Coordinates:
  * time        (time) datetime64[ns] 149kB 1973-01-01 1973-01-02 ... 2024-02-29
    step        timedelta64[ns] 8B 12:00:00
    surface     float64 8B 0.0
  * latitude    (latitude) float64 680B 62.0 61.5 61.0 60.5 ... 21.0 20.5 20.0
  * longitude   (longitude) float64 1kB 220.0 220.5 221.0 ... 305.0 305.5 306.0
    valid_time  (time) datetime64[ns] 149kB dask.array<chunksize=(365,), meta=np.ndarray>
Data variables:
    fwinx       (time, latitude, longitude) float32 1GB dask.array<chunksize=(365, 85, 173), meta=np.ndarray>
Attributes:
    GRIB_edition:            2
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-05-16T12:29 GRIB to CDM+CF via cfgrib-0.9.9...

In [91]:
df = pd.read_csv('../../data/UNSEEN/wildfire/cems/preprocessed/ERA5_yearly_max.csv')
df['time'] = pd.to_datetime(df['time'])
df = df[df['time'].dt.year != 2024]
df = df[(df['time'].dt.year >= 1987)]

#df = df[df['time'].dt.month.isin([5, 6, 7, 8, 9, 10])]

# Create a scatter plot with a trend line
fig = px.scatter(df, x='time', y='fwinx', 
                 hover_data={'time': '|%Y'},
                 trendline='ols')  # Add a linear trend line

# Add a line plot
fig.add_trace(go.Scatter(x=df['time'], y=df['fwinx'], mode='lines', name='FWINX'))

# Update layout
fig.update_layout(title='Fire Risk is Going Up Across North America<br><sub>Annual Maximum Fire Weather Index</sub>')
fig.update_yaxes(title_text='')
fig.update_xaxes(title_text='')
fig.update_layout(autosize=False, width=800)
fig.update_layout(showlegend=False)

# Show the plot
fig.show()

In [82]:
fig.write_html('../../figures/UNSEEN_wildfire/n_america_annual_max_fwi_2.html')
fig.write_image('../../figures/UNSEEN_wildfire/n_america_annual_max_fwi_2.png')

In [108]:
df = pd.read_csv('../../data/UNSEEN/wildfire/cems/preprocessed/canada/ERA5_yearly_max_British Columbia.csv')
df['time'] = pd.to_datetime(df['time'])

df = df[df['time'].dt.year < 2024]
df = df[(df['time'].dt.year >= 1987)]

# Create a scatter plot with a trend line
fig = px.scatter(df, x='time', y='fwinx', 
                 hover_data={'time': '|%Y'},
                 trendline='ols')  # Add a linear trend line

# Add a line plot
fig.add_trace(go.Scatter(x=df['time'], y=df['fwinx'], mode='lines', name='FWINX'))



# Update layout
fig.update_layout(title='Fire Risk is Going Up in BC<br><sub>Annual Maximum Fire Weather Index</sub>')
fig.update_yaxes(title_text='')
fig.update_xaxes(title_text='')
fig.update_layout(autosize=False, width=800)
fig.update_layout(showlegend=False)

# Show the plot
fig.show()

In [109]:
fig.write_html('../../figures/UNSEEN_wildfire/bc_annual_max_fwi.html')
fig.write_image('../../figures/UNSEEN_wildfire/bc_annual_max_fwi.png')

In [87]:
return_levels = pd.read_csv('../../data/UNSEEN/wildfire/cems/preprocessed/return_levels.csv')

return_levels.set_index('Unnamed: 0', inplace=True)
return_levels.index.name = 'Year'
df_long = pd.melt(return_levels.reset_index(), id_vars=['Year'], var_name='Return_Period', value_name='Return_Level')
df_long['Return_Period'] = pd.to_numeric(df_long['Return_Period'])
df_long.head()


,Year,Return_Period,Return_Level
0,1973,2,219.720029
1,2023,2,255.439611
2,1973,5,240.180758
3,2023,5,275.900339
4,1973,10,251.757157


In [89]:
# Create the plot
fig = px.line(
    df_long,
    x='Return_Period',
    y='Return_Level',
    color='Year',
    markers=True,
    title='',
    labels={
        'Return_Period': 'Return Period (years)',
        'Return_Level': 'Fire Weather Index Return Level'
    }
)

# Show the plot
fig.show()


In [67]:
ERA5.idxmax('time')

<xarray.Dataset> Size: 120kB
Dimensions:    (latitude: 85, longitude: 173)
Coordinates:
    step       timedelta64[ns] 8B 12:00:00
    surface    float64 8B 0.0
  * latitude   (latitude) float64 680B 62.0 61.5 61.0 60.5 ... 21.0 20.5 20.0
  * longitude  (longitude) float64 1kB 220.0 220.5 221.0 ... 305.0 305.5 306.0
Data variables:
    fwinx      (latitude, longitude) datetime64[ns] 118kB dask.array<chunksize=(85, 173), meta=np.ndarray>

In [14]:
from sklearn.metrics import r2_score

# Calculate a linear regression
coeffs_linear = np.polyfit(df['time_int'], df['fwinx'], 1)

# Generate y-values for the linear regression line
df['trendline_linear'] = coeffs_linear[0] * df['time_int'] + coeffs_linear[1]

# Calculate R-squared for the quadratic trend line
r2_quadratic = r2_score(df['fwinx'], df['trendline'])

# Calculate R-squared for the linear trend line
r2_linear = r2_score(df['fwinx'], df['trendline_linear'])

print(f'R-squared for the quadratic trend line: {r2_quadratic}')
print(f'R-squared for the linear trend line: {r2_linear}')

R-squared for the quadratic trend line: 0.21822230981889001
R-squared for the linear trend line: 0.15715222027621412


In [13]:
fig.write_html('../../figures/UNSEEN_wildfire/n_america_annual_max_fwi.html')
fig.write_image('../../figures/UNSEEN_wildfire/n_america_annual_max_fwi.png')

In [12]:
import pandas as pd
import plotly.express as px

# Load the DataFrame
df = pd.read_csv('../../data/UNSEEN/wildfire/cems/preprocessed/canada/province_stats_monthly.csv')

# Convert the 'Month' column to datetime
df['Month'] = pd.to_datetime(df['Month'])

# Create the line plot
fig = px.line(df, x='Month', y='fwinx_max', color='Province', title='Max fwinx over time by Province')

# Show the plot
fig.show()

In [116]:
# Load the DataFrame
df = pd.read_csv('../../data/UNSEEN/wildfire/cems/preprocessed/canada/ERA5_extreme_fwi_days_Quebec.csv')

# Convert the 'Month' column to datetime
#df['Year'] = pd.to_datetime(df['Year'])

# Create the line plot
fig = px.line(df, x='Year', y='p95_count',  title='Annual Max fwinx Quebec')

# Show the plot
fig.show()